<a href="https://colab.research.google.com/github/yelenana/machine_learning/blob/main/Rybchynska_Olena_%22HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/ML/2_2

Mounted at /content/drive
customer_segmentation_train.csv  regression_data.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.impute import SimpleImputer


In [4]:
raw_df = pd.read_csv('drive/MyDrive/ML/2_2/customer_segmentation_train.csv',index_col=0)

In [5]:
raw_df.head(3)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B


In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


Цікаво подивитися наскільки рівномурно наші категорії представлені у початковому наборі даних

In [7]:
raw_df['Segmentation'].value_counts()

,count
Segmentation,
D,2268
A,1972
C,1970
B,1858


In [8]:
train_df, val_df = train_test_split(raw_df, test_size=0.2, random_state=42, stratify=raw_df['Segmentation'])

In [9]:
# Створюємо трен. і вал. набори
input_cols = list(train_df.columns)[1:-1]
target_col = 'Segmentation'

train_inputs, train_targets = train_df[input_cols], train_df[target_col]
val_inputs, val_targets = val_df[input_cols], val_df[target_col]

In [10]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes(include='object').columns.tolist()

In [11]:
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')

# Створюємо копії та заповнюємо пропуски
train_inputs_filled = train_inputs.copy()
val_inputs_filled = val_inputs.copy()

train_inputs_filled[numeric_cols] = num_imputer.fit_transform(train_inputs[numeric_cols])
train_inputs_filled[categorical_cols] = cat_imputer.fit_transform(train_inputs[categorical_cols])

# Для валідаційних даних використовуємо ТІЛЬКИ transform
val_inputs_filled[numeric_cols] = num_imputer.transform(val_inputs[numeric_cols])
val_inputs_filled[categorical_cols] = cat_imputer.transform(val_inputs[categorical_cols])

Бачимо, що тепер пропущених даних нема

In [12]:
train_inputs_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6454 entries, 465905 to 459407
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ever_Married     6454 non-null   object 
 1   Age              6454 non-null   float64
 2   Graduated        6454 non-null   object 
 3   Profession       6454 non-null   object 
 4   Work_Experience  6454 non-null   float64
 5   Spending_Score   6454 non-null   object 
 6   Family_Size      6454 non-null   float64
 7   Var_1            6454 non-null   object 
dtypes: float64(3), object(5)
memory usage: 453.8+ KB


In [13]:
val_inputs_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 462588 to 461114
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ever_Married     1614 non-null   object 
 1   Age              1614 non-null   float64
 2   Graduated        1614 non-null   object 
 3   Profession       1614 non-null   object 
 4   Work_Experience  1614 non-null   float64
 5   Spending_Score   1614 non-null   object 
 6   Family_Size      1614 non-null   float64
 7   Var_1            1614 non-null   object 
dtypes: float64(3), object(5)
memory usage: 113.5+ KB


In [14]:
train_inputs_filled.head(3)

,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
ID,,,,,,,,
465905,No,32.0,Yes,Artist,9.000000,Low,1.0,Cat_6
462903,Yes,72.0,Yes,Entertainment,2.671488,Average,2.0,Cat_6
467901,No,33.0,Yes,Entertainment,1.000000,Low,4.0,Cat_6


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [15]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder

In [16]:
X_resampling = train_inputs_filled.copy()
val_inputs_filled_encoded = val_inputs_filled.copy()
label_encoders = {}

In [17]:
# Кодуємо категоріальні ознаки в числа
for col in categorical_cols:
    le = LabelEncoder()
    X_resampling[col] = le.fit_transform(X_resampling[col].astype(str))
    label_encoders[col] = le
    val_inputs_filled_encoded[col] = le.transform(val_inputs_filled_encoded[col].astype(str))

In [18]:
# для методу SMOTENC необхідно отримати індекси категоріальних ознак
cat_feature_indices = [X_resampling.columns.get_loc(col) for col in categorical_cols]
cat_feature_indices

[0, 2, 3, 5, 7]

In [19]:
# апсемплингом зі SMOTENC
smote_nc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_train_smote, y_train_smote = smote_nc.fit_resample(X_resampling, train_targets)

In [20]:
# ресамплингом з SMOTE-Tomek
smote_tomek = SMOTETomek(smote=smote_nc, random_state=42)
X_train_tomek, y_train_tomek = smote_tomek.fit_resample(X_resampling, train_targets)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [21]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

In [23]:
final_preprocessor = ColumnTransformer(transformers=[
    ('scaler', MinMaxScaler(), train_inputs.columns)
])

In [24]:
def train_and_report(X_train, y_train, X_val, y_val, name):
  model = Pipeline(steps=[
        ('preprocessor', final_preprocessor),
        ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=42)))
    ])
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  print(f"\n{'='*30}\nЗвіт для моделі: {name}\n{'='*30}")
  print(classification_report(y_val, y_pred))
  return model

In [25]:
# 1. Тренуємо на оригінальних (незбалансованих) даних
train_and_report(X_resampling, train_targets, val_inputs_filled_encoded, val_targets, "Original Data")


Звіт для моделі: Original Data
              precision    recall  f1-score   support

           A       0.39      0.40      0.40       394
           B       0.39      0.07      0.12       372
           C       0.47      0.63      0.54       394
           D       0.59      0.80      0.68       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.43      1614
weighted avg       0.47      0.49      0.45      1614



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  Index(['Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience',
       'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(random_state=42,
                                                                  solver='liblinear')))])

In [26]:
train_and_report(X_train_smote, y_train_smote, val_inputs_filled_encoded, val_targets, "SMOTENC")


Звіт для моделі: SMOTENC
              precision    recall  f1-score   support

           A       0.40      0.40      0.40       394
           B       0.40      0.15      0.22       372
           C       0.48      0.62      0.54       394
           D       0.61      0.77      0.68       454

    accuracy                           0.50      1614
   macro avg       0.47      0.49      0.46      1614
weighted avg       0.48      0.50      0.47      1614



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  Index(['Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience',
       'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(random_state=42,
                                                                  solver='liblinear')))])

In [27]:
train_and_report(X_train_tomek, y_train_tomek, val_inputs_filled_encoded, val_targets, "SMOTE-Tomek")


Звіт для моделі: SMOTE-Tomek
              precision    recall  f1-score   support

           A       0.41      0.37      0.39       394
           B       0.37      0.16      0.22       372
           C       0.47      0.62      0.54       394
           D       0.61      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.48      0.46      1614
weighted avg       0.47      0.50      0.47      1614



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  Index(['Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience',
       'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(random_state=42,
                                                                  solver='liblinear')))])

**Висновок:** якщо подивитися на результати отримані по початкових даний в після балансування - то видно, що різниця незначна. Я думаю, що це пов'язано з тим, що у початкових даних не видно прям такого критичного перекосу по представленості класів

In [31]:
raw_df['Segmentation'].value_counts()/raw_df['Segmentation'].count()*100

,count
Segmentation,
D,28.111056
A,24.442241
C,24.417452
B,23.029251


- тобто вцілому можна говорити про баланс класів (клас B займає 23%, а лідер D — 28%)

Однак, всеодно для обох моделей, SMOTENC та  Smote-Tomek, спостерігається покращення загального показника  accuracy на 0.01, і обов'язково потрібно звернути увагу на Macro Average F1-score - обидві моделі досягли Macro F1-score = 0.46, що на 0.03 вище за показник моделі на оригінальних даних (0.43). Також використання методів ребалансування позитивно вплинуло на правильну ідентифікацію клієнтів класу B, адже їх застосування допомогли підняти Recall для класу B з 0.07 до 0.15–0.16

Стосовно пунктів завдання:

3.Напишіть, яку метрику ви обрали для порівняння моделей - більше звертала увагу на Macro Average F1-score, але враховувала контекст, який задають і всі інші показники також

4.Яка модель найкраща? - на базі отриманих результатів не 2 бачу можливості обрати між SMOTENC та Smote-Tomek - вони показали себе однаково добре, але звісно краще ніж модель побудована на незбалансованих даних

5.Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому? -
так, суттєвої різниці нема, через відсутніст значного перекосу у представленості даних, детально розписала обгрунтування вище